# Dimensionality Reduction - Text Data

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import Isomap, TSNE, MDS
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import ticker
import time
import re
from tqdm import tqdm

import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Dense, Dropout, Reshape, UpSampling2D, Conv2DTranspose, Conv2D, MaxPool2D, Flatten, \
                                    Embedding, LSTM, Dense, Dropout, Input, RepeatVector, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.backend import clear_session

import faiss
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.csgraph import shortest_path

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

import torch

from datasets import load_dataset
from transformers import BertTokenizer, TFBertModel
from sentence_transformers import SentenceTransformer

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Check for GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(physical_devices))

Num GPUs Available:  1


## Standford IMDB

In [2]:
# Define stop words set
stop_words = ENGLISH_STOP_WORDS

# Define the lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Remove HTML tags using regular expressions
    text = re.sub(r'<.*?>', ' ', text.lower())
    # lowercase and remove apostrophe-s at end of words
    text = re.sub("'s\\b", " ", text.lower())
    # replace single letters followed by a period with a space
    text = re.sub(r"\b[a-zA-Z]\.", " ", text)
    # remove punctuation (any character that is not a word character)
    text = re.sub(r"[^\w\s]"," ", text)
    # # split text into words
    # tokens = word_tokenize(text)
    # # perform lemmatization on words with only alphabets and remove stop words
    # tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() 
    #           and lemmatizer.lemmatize(token) not in stop_words]
    # return ' '.join(tokens)
    return text

# Load the IMDB dataset
ds = load_dataset("scikit-learn/imdb")

# Preprocess the text data for all splits
def preprocess_function(data):
    data['review'] = [preprocess_text(text) for text in data['review']]
    return data

ds = ds['train'].map(preprocess_function, batched=True)

In [3]:
# Convert text data into TF-IDF vectors and split data into train and test sets
X = ds['review']
y = np.array(list(map(lambda x: 1 if x == 'positive' else 0, ds['sentiment'])))

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [4]:
model = LogisticRegression(random_state=42,  max_iter=1000)
model.fit(X_train, y_train)

pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
print(acc)

0.9009


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Convert text data to sentence embeddings
X_train_embeddings = model.encode(X_train, convert_to_tensor=True, device=device)
X_test_embeddings = model.encode(X_test, convert_to_tensor=True, device=device)

# Convert embeddings to numpy arrays
X_train_vec = X_train_embeddings.cpu().numpy()
X_test_vec = X_test_embeddings.cpu().numpy()

In [ ]:
# Train a simple linear classifier (Logistic Regression)
classifier = LogisticRegression(random_state=42,  max_iter=1000)
classifier.fit(X_train_vec, y_train)

# Predict sentiment on test data
y_pred = classifier.predict(X_test_vec)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
# Load GloVe embeddings
def load_glove_embeddings(filepath):
    embeddings_index = {}
    with open(filepath, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

embeddings_index = load_glove_embeddings('glove.42B.300d.txt')
embedding_dim = 300

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
max_sequence_length = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_sequence_length,
                    trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.005),
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate the model
pred = (model.predict(X_test) > 0.5).astype("int32")
acc = accuracy_score(y_test, pred)
print(f"Accuracy: {acc:.4f}")

In [6]:
# Create TaggedDocuments for Doc2Vec
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(X)]

# Train the Doc2Vec model
model = Doc2Vec(vector_size=200, window=5, min_count=1, workers=4)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=100)

# Extract document vectors
doc_vectors = np.array([model.dv[i] for i in range(len(tagged_data))])

X_train_vectors, X_test_vectors, y_train, y_test = train_test_split(doc_vectors, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train_vectors, y_train)

pred = model.predict(X_test_vectors)
acc = accuracy_score(y_test, pred)
print(acc)

c:\users\jzkho\appdata\local\programs\python\python38\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


0.5549


In [ ]:
# Load GloVe embeddings
def load_glove_embeddings(filepath):
    embeddings_index = {}
    with open(filepath, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

embeddings_index = load_glove_embeddings('glove.42B.300d.txt')

# Compute the average GloVe embeddings for each review
def get_average_embeddings(text_tokens, embeddings_index, embedding_dim=300):
    embeddings = [embeddings_index[token] for token in text_tokens if token in embeddings_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(embedding_dim)

df['embeddings'] = df['cleaned_text'].apply(lambda x: get_average_embeddings(x, embeddings_index))

X = np.vstack(df['embeddings'].values)
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model = LogisticRegression(random_state=42,  max_iter=1000)
model.fit(X_train, y_train)

pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
print(acc)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

def embed_text(text, tokenizer, model, max_length=512):
    inputs = tokenizer(text, return_tensors='tf', max_length=max_length, truncation=True, padding='max_length')
    outputs = model(inputs)
    return np.mean(outputs.last_hidden_state[0].numpy(), axis=0)

# Embed the text data
X_train_embeddings = []
for text in tqdm(X_train):
    X_train_embeddings.append(embed_text(text, tokenizer, model))
X_train_embeddings = np.array(X_train_embeddings)

X_test_embeddings = []
for text in tqdm(X_test):
    X_test_embeddings.append(embed_text(text, tokenizer, model))
X_test_embeddings = np.array(X_test_embeddings)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)

embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

# Define a function to process data in smaller batches
def get_embeddings_in_batches(data, embedding_layer, batch_size=64):
    num_samples = data.shape[0]
    embedding_shape = (num_samples, max_sequence_length * embedding_dim)
    embeddings_array = np.zeros(embedding_shape)

    for start in tqdm(range(0, num_samples, batch_size)):
        end = min(start + batch_size, num_samples)
        batch_data = data[start:end]
        batch_embeddings = embedding_layer(batch_data).numpy().reshape((batch_data.shape[0], -1))
        embeddings_array[start:end] = batch_embeddings
        clear_session()  # Clear the Keras session to free memory

    return embeddings_array

# Get the embeddings for train and test data
X_train_embedded = get_embeddings_in_batches(X_train, embedding_layer, batch_size=64)
X_test_embedded = get_embeddings_in_batches(X_test, embedding_layer, batch_size=64)

X_train_embedded = csr_matrix(X_train_embedded)
X_test_embedded = csr_matrix(X_test_embedded)

In [ ]:
# Train logistic regression model
logistic_model = LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1)
logistic_model.fit(X_train_embedded, y_train)

# Evaluate the model
pred = logistic_model.predict(X_test_embedded)
acc = accuracy_score(y_test, pred)
print(f"Accuracy: {acc:.4f}")

### PCA

In [ ]:
# Perform PCA on IMDB dataset
pca_imdb = decomposition.PCA()
pca_imdb.fit(X_train_vec)
x_train_pca = pca_imdb.transform(X_train_vec)

In [ ]:
# Examine the cumulative variance explained by principal components
cumulative_variance_ratio = np.cumsum(pca_imdb.explained_variance_ratio_)
x_ticks = list(range(1,len(cumulative_variance_ratio)+1))
plt.plot(x_ticks, cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio by Principal Components')
plt.show()

In [ ]:
# Standardise data before performing PCA
mnist_scaler = StandardScaler()
x_train_scaled = mnist_scaler.fit_transform(X_train_vec)
x_test_scaled = mnist_scaler.transform(X_test_vec)

In [ ]:
# Perform PCA on IMDB dataset
pca_imdb = decomposition.PCA()
pca_imdb.fit(x_train_scaled)
x_train_pca = pca_imdb.transform(x_train_scaled)

# Examine the cumulative variance explained by principal components
cumulative_variance_ratio = np.cumsum(pca_imdb.explained_variance_ratio_)
x_ticks = list(range(1,len(cumulative_variance_ratio)+1))
plt.plot(x_ticks, cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio by Principal Components')
plt.show()

In [ ]:
# Perform PCA on IMDB dataset
pca_imdb = decomposition.PCA(n_components=2)
pca_imdb.fit(x_train_scaled)
x_train_pca = pca_imdb.transform(x_train_scaled)

In [ ]:
# Define label set and distinct colourmap
unique_labels = set(y_train)

def plot_2d_comparison(x, y, n_data, unique_labels, x_label, y_label, title):
    plt.figure(figsize=(8,6))
    for i in unique_labels:
        plt.scatter(x[:n_data][y[:n_data] == i, 0], 
                    x[:n_data][y[:n_data] == i, 1], 
                    label=i, alpha=0.6)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Visualise the first two principal components
plot_2d_comparison(x_train_pca, y_train, 1500, unique_labels, 'PC1', 'PC2', 'PCA - IMDB')

### Isomap

In [ ]:
# Reduce the dimensionality of the dataset to two-dimensional using Isomap
isomap_imdb = Isomap(n_components=2, n_neighbors=50) 
# Fit the isomap algorithn using the first 30,000 data points to reduce compute time
x_train_isomap = isomap_imdb.fit_transform(X_train_vec[:10000]) 

# Visualise the 2d representation of the dataset
plot_2d_comparison(x_train_isomap, y_train, 1500, unique_labels, 
               'Dim1', 'Dim2', 'Isomap - IMDB')

### Geodesic distance + MDS

In [ ]:
# JIT-compiled function for constructing the graph
@jit(nopython=True)
def construct_graph(indices, distances, n_neighbors, n_samples):
    graph = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(1, n_neighbors):  # start from 1 to skip self-loop
            graph[i, indices[i, j]] = distances[i, j]
            graph[indices[i, j], i] = distances[i, j]
    return graph

# JIT-compiled function for constructing the expanded graph
@jit(nopython=True)
def construct_expanded_graph(indices, distances, neigh_graph, n_neighbors, n_train, n_samples):
    n_total = n_train + n_samples
    graph = np.zeros((n_total, n_total))
    graph[:n_train, :n_train] = neigh_graph
    for i in range(n_samples):
        for j in range(1, n_neighbors):  # start from 1 to skip self-loop
            graph[n_train+i, indices[i, j]] = distances[i, j]
            graph[indices[i, j], n_train+i] = distances[i, j]
    return graph

def compute_geodesic_distances(data, n_neighbors=10, training=False, faiss_index=None, neigh_graph=None):
    print(f"Memory usage at start: {memory_usage_psutil()} MB")

    if training:
        # Use FAISS to find approximate nearest neighbors
        d = data.shape[1]
        faiss_index = faiss.IndexFlatL2(d)
        faiss_index.add(data)
        distances, indices = faiss_index.search(data, n_neighbors)

        # Construct sparse neighborhood graph
        n_samples = data.shape[0]
        print(f"Memory usage before constructing graph: {memory_usage_psutil()} MB")
        graph = construct_graph(indices, distances, n_neighbors, n_samples)
        print(f"Memory usage after constructing graph: {memory_usage_psutil()} MB")

        # Compute geodesic distances using the shortest path algorithm
        print(f"Memory usage before shortest path: {memory_usage_psutil()} MB")
        geodesic_distances = shortest_path(csgraph=graph, method='D', directed=False)
        print(f"Memory usage after shortest path: {memory_usage_psutil()} MB")
    else:
        # Use trained FAISS index to find approximate nearest neighbors
        n_train = faiss_index.ntotal
        n_samples = data.shape[0]
        print(f"Memory usage before searching: {memory_usage_psutil()} MB")
        distances, indices = faiss_index.search(data, n_neighbors)

        # Construct sparse neighborhood graph
        print(f"Memory usage before constructing expanded graph: {memory_usage_psutil()} MB")
        graph = construct_expanded_graph(indices, distances, neigh_graph, n_neighbors, n_train, n_samples)
        print(f"Memory usage after constructing expanded graph: {memory_usage_psutil()} MB")

        # Compute geodesic distances using the shortest path algorithm for all data
        print(f"Memory usage before shortest path: {memory_usage_psutil()} MB")
        geodesic_distances = shortest_path(csgraph=graph, method='D', directed=False)
        print(f"Memory usage after shortest path: {memory_usage_psutil()} MB")

        # Extract the geodesic distances matrix for only new data
        geodesic_distances = geodesic_distances[n_train:, n_train:]
    
    # Explicitly call garbage collector
    gc.collect()
    print(f"Memory usage after garbage collection: {memory_usage_psutil()} MB")

    return geodesic_distances, faiss_index, graph

start_time = time.time()
# Compute geodesic distances for mnist dataset (training)
n_neighbors = 5
geodesic_distances, faiss_index, neigh_graph = compute_geodesic_distances(X_train, n_neighbors=n_neighbors, training=True)
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Time taken to run the function: {elapsed_time} seconds")
# 15000: 117s

print(geodesic_distances.shape)
print(faiss_index.ntotal)

In [ ]:
start_time = time.time()
n_neighbors = 10
data = X_train

# Use FAISS to find approximate nearest neighbors
d = data.shape[1]
faiss_index = faiss.IndexFlatL2(d)
faiss_index.add(data)
distances, indices = faiss_index.search(data, n_neighbors)

# Construct sparse neighborhood graph
n_samples = data.shape[0]
graph = np.zeros((n_samples, n_samples))
for i in range(n_samples):
    for j in range(1, n_neighbors):  # start from 1 to skip self-loop
        graph[i, indices[i, j]] = distances[i, j]
        graph[indices[i, j], i] = distances[i, j]

graph = csr_matrix(graph)

# Compute geodesic distances using the shortest path algorithm
geodesic_distances = shortest_path(csgraph=graph, method='D', directed=False)

# Calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time taken to run the function: {elapsed_time} seconds")

In [ ]:
# Save the numpy array to a CSV file
np.savetxt("geo_dist_imdb.csv", geodesic_distances, delimiter=",", fmt="%d")

In [ ]:
# Load data from CSV file
geodesic_distances = np.loadtxt("geo_dist_imdb.csv", delimiter=",", dtype=np.float32)

# Verify the loaded data
print("Loaded data shape:", geodesic_distances.shape)

### t-SNE

In [ ]:
# Reduce the dimensionality of the dataset to two-dimensional using t-SNE
tsne_imdb = TSNE(n_components=2, init='random') 
x_train_tsne = tsne_imdb.fit_transform(X_train_vec[:10000])

# Visualise the 2d representation of the dataset
plot_2d_comparison(x_train_tsne, y_train, 1500, unique_labels,
               'Dim1', 'Dim2', 't-SNE - MNIST')

### Autoencoder

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])
word_index = tokenizer.word_index

# Pad sequences to ensure uniform input size
maxlen = 1000
X = pad_sequences(sequences, maxlen=maxlen)

# Prepare labels
y = labels

In [ ]:
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Load GloVe embeddings
glove_file = 'glove.6B.50d.txt'  # Adjust path and file according to your download
embeddings_index = load_glove_embeddings(glove_file)

def create_embedding_matrix(word_index, embeddings_index, embedding_dim):
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for word, i in word_index.items():
        if i >= num_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_dim = 50
embedding_matrix = create_embedding_matrix(word_index, embeddings_index, embedding_dim)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

embeddings = Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False)(X)

In [ ]:
# Compute the encodings after training
untrained_encodings_mnist = mnist_encoder(x_test).numpy()

# Visualise the trained latent encodings
plot_2d_mnist(untrained_encodings_mnist, y_test, 3000, unique_labels, 
               'Dim1', 'Dim2', 'Autoencoder - MNIST')

In [ ]:
# Create Dataset objects for train and test sets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, x_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, x_test))

# Shuffle and batch the datasets
shuffle_buffer_size = 1000
batch_size = 64
train_dataset = train_dataset.shuffle(shuffle_buffer_size, seed).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Create an EarlyStopping callback to terminate training if the validation total loss doesn't immprove after 10 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# Compile and fit the model
mnist_autoencoder.compile(optimizer='Adam', loss='mean_squared_error')
history_mnist = mnist_autoencoder.fit(train_dataset, validation_data=test_dataset, epochs=100, callbacks=early_stopping, verbose=0)

In [ ]:
# Print final training and validation loss (MSE)
final_train_loss = history_mnist.history['loss'][-1]
final_val_loss = history_mnist.history['val_loss'][-1]
print(f"Final Training Loss: {final_train_loss:.4f}, Final Validation Loss: {final_val_loss:.4f}")

# Examine the training and validation loss (MSE) over epochs
plt.plot(history_mnist.history['loss'], label='train')
plt.plot(history_mnist.history['val_loss'], label='val')
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("Loss vs epoch")
plt.legend()

In [ ]:
# Compute the encodings after training
trained_encodings_mnist = mnist_encoder(x_test).numpy()

# Visualise the trained latent encodings
plot_2d_mnist(trained_encodings_mnist, y_test, 1000, unique_labels, 
               'Dim1', 'Dim2', 'Autoencoder - MNIST')

In [ ]:
np.random.seed(5)
inx = np.random.choice(x_test.shape[0], 10, replace=False)
reconstructed_mnist_images = mnist_autoencoder(x_test[inx])

f, axs = plt.subplots(2, 10, figsize=(15, 4))
for j in range(10):
    axs[0, j].imshow(x_test[inx][j], cmap='binary')
    axs[1, j].imshow(reconstructed_mnist_images[j].numpy().squeeze(), cmap='binary')
    axs[0, j].axis('off')
    axs[1, j].axis('off')
plt.show()

### GCAE

In [ ]:
class GCAETrainer(Model):
    """
    A trainer class for a GCAE that handles model training and evaluation.
    """
    def __init__(self, dataset, input_dim=(None,None,1), embedding_dim=2, n_neighbors=10, alpha=0.01, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.dataset = dataset
        self.n_neighbors = n_neighbors
        self.gcae = get_cnn_autoencoder(self.input_dim, self.embedding_dim, self.dataset)
        self.encoder = self.gcae.get_layer(f'{self.dataset}_cnn_encoder')
        self.alpha = alpha

        # FAISS index and training data
        self.faiss_index = None
        self.neigh_graph = None
        self.train_geo_dist = None
        self.train_geo_dist_max = None
        self.test_geo_dist = None

        # Define the loss metrics to track and log
        self.total_loss_metric = Mean(name='total_loss')
        self.reconstruction_loss_metric = Mean(name='reconstruction_loss')
        self.geodesic_loss_metric = Mean(name='geodesic_loss')

    @property
    def metrics(self):
        """
        Returns a list of metrics used in training and evaluation.
        """
        return [self.total_loss_metric, self.reconstruction_loss_metric, self.geodesic_loss_metric]

    def compute_geodesic_distances(self, data, training=False):
    
        if training:
            
            # Use FAISS to find approximate nearest neighbors
            d = data.shape[1]
            faiss_index = faiss.IndexFlatL2(d)
            faiss_index.add(data)
            self.faiss_index = faiss_index
            distances, indices = faiss_index.search(data, self.n_neighbors)
        
            # Construct sparse neighborhood graph
            n_samples = data.shape[0]
            graph = np.zeros((n_samples, n_samples))
            for i in range(n_samples):
                for j in range(1, self.n_neighbors):  # start from 1 to skip self-loop
                    graph[i, indices[i, j]] = np.sqrt(distances[i, j])
                    graph[indices[i, j], i] = np.sqrt(distances[i, j])          
            graph = csr_matrix(graph)
            self.neigh_graph = graph
        
            # Compute geodesic distances using the shortest path algorithm
            geodesic_distances = shortest_path(csgraph=graph, method='D', directed=False)
            geodesic_distances = np.float32(geodesic_distances)
            self.train_geo_dist = geodesic_distances
            self.train_geo_dist_max = np.max(geodesic_distances)
            
        else:

            # Use trained FAISS index to find approximate nearest neighbors
            n_train = self.faiss_index.ntotal
            n_samples = tf.shape(data)[0]
            n_total = n_train + n_samples
            distances, indices = self.faiss_index.search(data, self.n_neighbors)
    
            # Construct sparse neighborhood graph
            graph = np.zeros((n_total, n_total))
            graph[:n_train, :n_train] = self.neigh_graph
            for i in range(n_samples):
                for j in range(1, self.n_neighbors):  # start from 1 to skip self-loop
                    graph[n_train+i, indices[i, j]] = np.sqrt(distances[i, j])
                    graph[indices[i, j], n_train+i] = np.sqrt(distances[i, j])
            graph = csr_matrix(graph)
            
            # Compute geodesic distances using the shortest path algorithm for all data
            geodesic_distances = shortest_path(csgraph=graph, method='D', directed=False)
    
            # Extract the geodesic distances matrix for only new data
            geodesic_distances = geodesic_distances[n_train:, n_train:]
            geodesic_distances = np.float32(geodesic_distances)
            self.test_geo_dist = geodesic_distances
            
        return geodesic_distances
    
    def compute_geodesic_loss(self, z, geodesic_distances, batch_indices):
        """
        Computes geodesic distance loss.
        """
        # Calculate the pairwise Euclidean distances in the latent space
        z_distances = tf.norm(z[:, tf.newaxis] - z, axis=2)
        # Calculate the geodesic distance loss
        batch_indices_np = batch_indices.numpy()
        geodesic_distances_batch = geodesic_distances[np.ix_(batch_indices_np, batch_indices_np)]
        # geodesic_distances_batch = tf.gather(tf.gather(geodesic_distances, batch_indices, axis=0), batch_indices, axis=1)
        distance_loss = tf.reduce_mean((z_distances - geodesic_distances_batch) ** 2) / self.train_geo_dist_max
        return distance_loss
        
    def _get_losses(self, x, batch_indices, training=False):
        """
        Computes model losses from inputs.
        """
        reconstructions = self.gcae(x, training=training)
        latent = self.encoder(x)
        # Compute the reconstruction loss
        x = tf.expand_dims(x, axis=-1)
        reconstruction_loss = (
            tf.reduce_mean((x - reconstructions) ** 2)
        )
        # Compute the geodesic loss
        if training:
            geodesic_loss = self.compute_geodesic_loss(z=latent, geodesic_distances=self.train_geo_dist, 
                                                       batch_indices=batch_indices)
        else:    
            geodesic_loss = self.compute_geodesic_loss(z=latent, geodesic_distances=self.test_geo_dist, 
                                                       batch_indices=batch_indices)
        # Cmpute the total loss
        total_loss = reconstruction_loss + self.alpha * geodesic_loss
        return total_loss, reconstruction_loss, geodesic_loss

    def train_step(self, data):
        """
        Performs one training step using a single batch of data.
        """
        x, batch_indices = data
        with tf.GradientTape() as tape:
            total_loss, reconstruction_loss, geodesic_loss = self._get_losses(x, batch_indices, training=True)
        grads = tape.gradient(total_loss, self.gcae.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.gcae.trainable_variables))

        # Update loss metrics
        self.total_loss_metric.update_state(total_loss)
        self.reconstruction_loss_metric.update_state(reconstruction_loss)
        self.geodesic_loss_metric.update_state(geodesic_loss)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        """
        Evaluates the model using a single batch of data.
        """
        x, batch_indices = data
        total_loss, reconstruction_loss, geodesic_loss = self._get_losses(x, batch_indices, training=False)

        # Update loss metrics
        self.total_loss_metric.update_state(total_loss)
        self.reconstruction_loss_metric.update_state(reconstruction_loss)
        self.geodesic_loss_metric.update_state(geodesic_loss)
        return {m.name: m.result() for m in self.metrics}


# Define the gcae trainer
gcae_trainer = GCAETrainer(input_dim=input_shape, embedding_dim=encoded_dim, n_neighbors=10, dataset='mnist')

# Ensure eager execution
tf.config.run_functions_eagerly(True)

# # Generate Geodesic distance matrix for train and validation data
# train_start_time = time.time()
# train_geo_dist = gcae_trainer.compute_geodesic_distances(X_train, training=True)
# train_end_time = time.time()
# print(f"Time taken to compute geodesic distances for train data: {train_end_time - train_start_time} seconds") # 2393.357335090637 seconds

# # Save the numpy array to a CSV file
# np.savetxt("mnist_train_geo_dist.csv", gcae_trainer.train_geo_dist, delimiter=",", fmt="%d")

# val_start_time = time.time()
# val_geo_dist = gcae_trainer.compute_geodesic_distances(X_test, training=False)
# val_end_time = time.time()
# print(f"Time taken to compute geodesic distances for val data: {val_end_time - val_start_time} seconds") # 3650.7034978866577 seconds

# # Save the numpy array to a CSV file
# np.savetxt("mnist_test_geo_dist.csv", gcae_trainer.test_geo_dist, delimiter=",", fmt="%d")

In [ ]:
# Load data from CSV file
train_geo_dist = np.loadtxt("mnist_train_geo_dist.csv", delimiter=",", dtype=np.float32)
val_geo_dist = np.loadtxt("mnist_test_geo_dist.csv", delimiter=",", dtype=np.float32)

# Verify the loaded data
print("Loaded MNIST train geo dist shape:", train_geo_dist.shape)
print("Loaded MNIST test geo dist shape:", val_geo_dist.shape)

In [ ]:
gcae_trainer.train_geo_dist = train_geo_dist
gcae_trainer.train_geo_dist_max = np.max(train_geo_dist)
gcae_trainer.test_geo_dist = val_geo_dist

In [ ]:
# Convert data to TensorFlow Dataset
batch_size = 64
train_data = tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32), np.arange(len(x_train))))
train_data = train_data.batch(batch_size)
val_data = tf.data.Dataset.from_tensor_slices((x_test.astype(np.float32), np.arange(len(x_test))))
val_data = val_data.batch(batch_size)

# Create an EarlyStopping callback to terminate training if the validation total loss doesn't immprove after 10 epochs
early_stopping = EarlyStopping(monitor='val_total_loss', patience=10, mode='min', restore_best_weights=True)

# Train the autoencoder over 100 epochs
gcae_trainer.compile(optimizer='adam')
history_gc_mnist = gcae_trainer.fit(train_data, validation_data=val_data, epochs=100, callbacks=early_stopping, verbose=0)

# Print final training and validation loss (MSE)
final_train_loss = history_gc_mnist.history['total_loss'][-1]
final_val_loss = history_gc_mnist.history['val_total_loss'][-1]
print(f"Final Training Loss: {final_train_loss:.4f}, Final Validation Loss: {final_val_loss:.4f}")

# Examine the training and validation loss (MSE) over epochs
plt.figure(figsize=(20,4))
plt.subplot(1,3,1)
plt.plot(history_gc_mnist.history['total_loss'], label='train')
plt.plot(history_gc_mnist.history['val_total_loss'], label='val')
plt.xlabel("Epoch")
plt.ylabel("Total Loss (MSE + Geodesic Loss)")
plt.title("Total Loss vs epoch")
plt.legend()

plt.subplot(1,3,2)
plt.plot(history_gc_mnist.history['reconstruction_loss'], label='train')
plt.plot(history_gc_mnist.history['val_reconstruction_loss'], label='val')
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("Reconstruction Loss vs epoch")
plt.legend()

plt.subplot(1,3,3)
plt.plot(history_gc_mnist.history['geodesic_loss'], label='train')
plt.plot(history_gc_mnist.history['val_geodesic_loss'], label='val')
plt.xlabel("Epoch")
plt.ylabel("Geodesic Loss")
plt.title("Geodesic Loss vs epoch")
plt.legend()
plt.show()

In [ ]:
mnist_gc_encoder = gcae_trainer.gcae.get_layer('mnist_cnn_encoder')

# Compute the encodings after training
trained_encodings_gc_mnist = mnist_gc_encoder(x_test).numpy()

# Visualise the trained latent encodings
plot_2d_mnist(trained_encodings_gc_mnist, y_test, 1000, unique_labels, 
               'Dim1', 'Dim2', 'GCAE - MNIST')

In [ ]:
mnist_gc_autoencoder = gcae_trainer.gcae

np.random.seed(5)
inx = np.random.choice(x_test.shape[0], 10, replace=False)
reconstructed_mnist_images_gc = mnist_gc_autoencoder(x_test[inx])

f, axs = plt.subplots(2, 10, figsize=(15, 4))
for j in range(10):
    axs[0, j].imshow(x_test[inx][j], cmap='binary')
    axs[1, j].imshow(reconstructed_mnist_images_gc[j].numpy().squeeze(), cmap='binary')
    axs[0, j].axis('off')
    axs[1, j].axis('off')
plt.show()

### Logistic Regression

In [ ]:
# Train logistic classifier using original data
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# Validate accuracy using test data
pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
print(acc)

In [ ]:
seed = 42
input_shape = (28, 28, 1)
dims = [2, 10, 20, 50, 100]
accs = []

for d in dims:
    # Build and train the CNN autoencoder for the MNIST dataset
    mnist_autoencoder = get_cnn_autoencoder(input_shape, d, 'mnist')
    mnist_encoder = mnist_autoencoder.get_layer('mnist_cnn_encoder')
    mnist_autoencoder.compile(optimizer='Adam', loss='mean_squared_error')
    history_mnist = mnist_autoencoder.fit(train_dataset, validation_data=test_dataset, epochs=100, callbacks=early_stopping, verbose=0)

    # Generate encodings for train data
    trained_mnist_encodings = mnist_encoder(x_train).numpy()

    # Train logistic classifier using standard autoencoder's embeddings
    model = LogisticRegression(random_state=seed, max_iter=1000)
    model.fit(trained_mnist_encodings, y_train)

    # Generate encodings for test data
    predicted_mnist_encodings = mnist_encoder(x_test).numpy()
    
    # Validate classification accuracy on test data
    pred = model.predict(predicted_mnist_encodings)
    acc = accuracy_score(y_test, pred)
    accs.append(acc)
    print(f'Accuracy with {d} dimensions: {acc}')

In [ ]:
seed = 42
input_shape = (28, 28, 1)
dims = [2, 10, 20, 50, 100]
accs = []

for d in dims:
    # Build and train the CNN autoencoder for the MNIST dataset
    mnist_autoencoder = get_cnn_autoencoder(input_shape, d, 'mnist')
    mnist_encoder = mnist_autoencoder.get_layer('mnist_cnn_encoder')
    mnist_autoencoder.compile(optimizer='Adam', loss='mean_squared_error')
    history_mnist = mnist_autoencoder.fit(train_dataset, validation_data=test_dataset, epochs=100, callbacks=early_stopping, verbose=0)

    # Generate encodings for train data
    trained_mnist_encodings = mnist_encoder(x_train).numpy()

    # Train logistic classifier using standard autoencoder's embeddings
    model = LogisticRegression(random_state=seed, max_iter=1000)
    model.fit(trained_mnist_encodings, y_train)

    # Generate encodings for test data
    predicted_mnist_encodings = mnist_encoder(x_test).numpy()
    
    # Validate classification accuracy on test data
    pred = model.predict(predicted_mnist_encodings)
    acc = accuracy_score(y_test, pred)
    accs.append(acc)
    print(f'Accuracy with {d} dimensions: {acc}')